In [54]:
import numpy as np
from itertools import izip
from math import *

In [55]:
R = 8.314 # [kg/(J.K)]

In [56]:
class Component():
    def __init__(self, a, b):
        self.a = a
        self.b = b

In [62]:
class WanDerWaalsEos():
    def __init__(self, components, binary_interaction):
        self.components = components
        self.binary_interaction = binary_interaction
        
    def set_molar_fractions(self, molar_fractions):
        self.molar_fractions = molar_fractions
    
    def update_eos_coefficients(self):
        self.b = 0.0
        self.a = 0.0
        for i, (y_i, comp_i) in enumerate(izip(self.molar_fractions, self.components)):
            self.b += y_i * comp_i.b
            for j, (y_j, comp_j) in enumerate(izip(self.molar_fractions, self.components)):
                delta = self.binary_interaction[i, j]
                a_ij = delta * sqrt(comp_i.a * comp_j.a)
                self.a += y_i * y_j * a_ij
        
    def compute_fugacity(self, index, temperature):
        component = self.components[i]
        T = temperature
        
        
        

In [83]:
comp_a = Component(a=0.55237, b=3.0412e-5)
comp_b = Component(a=0.42548, b=3.738e-5)
components_list = [comp_a, comp_b]

delta_ij = 0.47126/sqrt(comp_a.a * comp_b.a)

binary_interaction = np.array(
    [[1.000000, delta_ij],
     [delta_ij, 1.000000]]
)

In [84]:
eos = WanDerWaalsEos(components_list, binary_interaction)

In [85]:
molar_fractions = np.array([0.9, 0.1])

In [86]:
eos.set_molar_fractions(molar_fractions)
eos.update_eos_coefficients()

print eos.a
print eos.b

0.5365013
3.11088e-05
